<a href="https://colab.research.google.com/github/vsoni03/AI-projects/blob/main/Personal_Fine_Tuning_LLMs_with_Hugging_Face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-Tuning LLMs with Hugging Face

## Step 1: Installing and importing the libraries

In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/2

In [2]:
!pip uninstall -y torch torchvision torchaudio bitsandbytes
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118  # Colab typically uses CUDA 11.8
!pip install bitsandbytes

Found existing installation: torch 2.5.1+cu124
Uninstalling torch-2.5.1+cu124:
  Successfully uninstalled torch-2.5.1+cu124
Found existing installation: torchvision 0.20.1+cu124
Uninstalling torchvision-0.20.1+cu124:
  Successfully uninstalled torchvision-0.20.1+cu124
Found existing installation: torchaudio 2.5.1+cu124
Uninstalling torchaudio-2.5.1+cu124:
  Successfully uninstalled torchaudio-2.5.1+cu124
Found existing installation: bitsandbytes 0.40.2
Uninstalling bitsandbytes-0.40.2:
  Successfully uninstalled bitsandbytes-0.40.2
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 83.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 3.9 MB/s eta 0:

In [3]:
!pip install huggingface_hub
!nvidia-smi

Tue Feb 18 16:52:29 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   31C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
import torch
from trl import SFTTrainer
from peft import LoraConfig
from datasets import load_dataset
from transformers import (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline)

## Step 2: Loading the model

In [5]:
# load in 4 bit quantization
llama_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path = "aboonaji/llama2finetune-v2",
                                                   quantization_config = BitsAndBytesConfig(load_in_4bit = True,
                                                                                            bnb_4bit_compute_dtype = getattr(torch, "float16"),
                                                                                            bnb_4bit_quant_type = "nf4"))
# dont store precomputed layhers
llama_model.config.use_cache = False
# deactivate the the more acccurate computationa and will speed up computation
llama_model.config.pretraining_tp = 1


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

This loads a fine-tunned Llama 2 model from hugging face. It enables a 4-bit quantization reducing model size and memory usage. It also specificed that the computation is done in float15 which balances speed and precision. It also uses normal float 4 which is an optimized quantization technique that provides better accuracy than the standard 4-bit quantization. There is also use chache which used to store precomputed hidden states during interference. It forces the model to recompute the previous states instead of relying on cached ones. It helps to reduce memory usage but slows down generation/ The pretraining parallism is set to 1 because it ensures a single partition whcih speed up inference but more memory helps when model is on one gpu rather across multiple.

## Step 3: Loading the tokenizer

In [6]:
llama_tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path = "aboonaji/llama2finetune-v2", trust_remote_code = True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

This loads the pretrained toklenizer associated with the fine-tunned LLama 2 model. It allows for execution of custom tokenizer code if it exists in the remote model repository-useful when the tokenizer has custom logic not part of hugging face. It has a predefined padding token for when padding is needed ensures all the same length without introducing out of vocab tokens. It adds to the end - right side of shorter sequences.

## Step 4: Setting the training arguments

In [7]:
training_arguments =  TrainingArguments(output_dir = "./results", per_device_train_batch_size = 4, max_steps = 100)

training arguments is where it configurations of various training parameters when fine-tuning a model. The output directory where the training outputs will be saved (logs, checkpoints, model, weights). It will be saved in a folder named results in the current working directory. Helps resuming training or loading the best model later. The batch size per gpu during training is 4. The maximum number of training steps each step corresponds to one foward + backward pass for a single batch. The training stops after 100 updates - updates the models weights.

## Step 5: Creating the Supervised Fine-Tuning trainer

In [8]:
llama_sft_trainer = SFTTrainer(model = llama_model,
                               args = training_arguments,
                               train_dataset =load_dataset(path = "aboonaji/wiki_medical_terms_llam2_format", split = "train"),
                               tokenizer = llama_tokenizer,
                               peft_config = LoraConfig(task_type = "CAUSAL_LM", r = 64, lora_alpha = 16, lora_dropout = 0.1),
                               dataset_text_field = "text")

wiki_medical_terms_llam2.jsonl:   0%|          | 0.00/54.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6861 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/6861 [00:00<?, ? examples/s]

The code is using the SFTTrainer supervised fine-ttuning trainer to fine tune a llama 2 model using lora (low rank adaptation) on dataset of medical terms. The lora is a parameter-efficent fine tuning technique designed without modifying all model weights. Instead of updates all weight matrices. It injects small, low rank matrices into to reduce computation and memory overhead. The llama 2 model loaded earlier, args used are the training arguments. The train dataset loads a dataset from hugging face and use strain for fine tuning/ The tokenizer is text properly tokenizer created before. The pfte configures lora - a lightweight fine-tuning method that reduces Vram usage and speds up training. It fine tunes a casual language model, rank of LORA matrices, lora alpha is scaling factor for the lora layers affects learning stability, and adds dropout 10 percent dropped to prevent overfitting. The higher rank controls the rank - more expressive adaptation.

sfttrainer:
- loads the model with lora setting (modifies model weight)
- handles dataset loading and preprocessing
- trains only the lora parameters instead of full model
- applies optimization for fast and memory efficent fine-tuning.


## Step 6: Training the model

In [ ]:
llama_sft_trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

Just trains the model and doing the steps and using the tokenizer for llama. The tokenizer converts raw text into tokens that the model can undertsand and process - it crucial component in NLP models.



## Step 7: Chatting with the model

In [ ]:
user_prompt = ""
text_generation_pipeline = pipeline(task = "text-generation", model = llama_model, tokenizer = llama_tokenizer, max_length = 300)
# input of text generation
model_answer = text_generation_pipeline(f"<s>[INST]{user_prompt} [/INST]")
print(model_answer[0]["generated_text"])

Fine-tuned LLM to chat about medical terms and it is trained for this and chat about medical terms.

### What does this model do? ###

It starts with a general Llama 2, a general purpose language model. It trains on medical terminology dataset to improve domain knoeledge. It uses 4-quantization to run smaller gpus. It uses lora to train only small low-rank matrices making fine-tuning fast/ It uses SFTtrainer to train and optimize data processing. Produces a specialize medical chatbot or assistant that answers health-related queries accurately.




1.   Loads a Pre-trained LLama Model

It starts with pretrained llama model
Base model has been trained on a general text

2.   Uses 4-bit quantization to save memory

Instead of using full precision, loads the model in 4-bit precision.
Allows the model to run a consumer-grade cpus

3. Applies Lora for efficent fine-tuning

Instead of modifying all weights, adds a small rank matrices that learn only the medical domain knowledge. This makes training lightweight and fast while keeping the original llama model intact.

4. Load Medical Data Set for fine tuning

5. Fine-tunes using SFT Trainer

6. Generates medical specfic responses

After fine-tuning, the model generate medical related text with greater accuracy.

🔹 Example Before Fine-Tuning (Base Model)
User: What are the symptoms of diabetes?
LLaMA 2 (Base Model): Diabetes is a metabolic disorder that affects blood sugar levels. (Generic answer)

🔹 Example After Fine-Tuning (Fine-Tuned Model)
User: What are the symptoms of diabetes?
Fine-Tuned LLaMA 2 Model: Diabetes symptoms include excessive thirst, frequent urination, unexplained weight loss, fatigue, blurred vision, and slow wound healing. (More specific and medically accurate)

7. Model generate detailed and accurate medical responses.


